In [49]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# 数据预处理
def encode_categorical(df):
    label_encoder = LabelEncoder()
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = label_encoder.fit_transform(df[column])
        elif df[column].dtype == 'float64' or df[column].dtype == 'int64':
            df[column] = df[column].fillna(df[column].mean())
    return df

df1 = pd.read_csv('train.csv')
df1 = encode_categorical(df1)


In [50]:
from sklearn.model_selection import train_test_split
import torch.utils.data as Data
import torch

x = df1.iloc[:,1:-1]
y= df1.iloc[:,-1:]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2004)

x_train_tensor = torch.FloatTensor(x_train.values)
y_train_tensor = torch.FloatTensor(y_train.values)
x_test_tensor = torch.FloatTensor(x_test.values)
y_test_tensor = torch.FloatTensor(y_test.values)

train_dataset = Data.TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = Data.TensorDataset(x_test_tensor, y_test_tensor)

train_dataloader = Data.DataLoader(train_dataset,batch_sizes,shuffle=True)
test_dataloader = Data.DataLoader(test_dataset,batch_sizes,shuffle=True)


In [59]:
# 构建模型
import torch.nn as nn
class LinearNet(nn.Module):
    def __init__(self,in_features,out_features):
        super().__init__()
        self.linear_1 = nn.Linear(in_features, 40)  # nn.Module
        self.linear_2 = nn.Linear(40, 10)
        self.linear_3 = nn.Linear(10, out_features)
        self.activation = nn.Sigmoid()
    def forward(self,x):
        y = self.activation(self.linear_1(x))
        y = self.activation(self.linear_2(y))
        y = self.activation(self.linear_3(y))
        return y

In [60]:
# 设置优化器和损失函数
import torch.optim as optim
loss = nn.MSELoss()
model = LinearNet(79,1)
optimizer = optim.Adam(model.parameters(),lr=1e-3)


In [61]:
# 训练模型
def train_model(model,dataloader):
    model.train()
    
    total_loss = 0
    for idx,(x,y) in enumerate(dataloader):
        y_pred = model(x)
        cur_loss = loss(y_pred,y)
        optimizer.zero_grad()
        cur_loss.backward()
        optimizer.step()
        
        total_loss += cur_loss.item()
    print(f"train loss:{total_loss/len(train_dataset)}")
    

In [55]:
Epoch = 100
for i in range(Epoch):
    train_model(model,train_dataloader)

train loss:2385596601.8630137
train loss:2385596633.4246573
train loss:2385596615.890411
train loss:2385596601.8630137
train loss:2385596600.109589
train loss:2385596600.109589
train loss:2385596628.1643834
train loss:2385596612.3835616
train loss:2385596601.8630137
train loss:2385596621.150685
train loss:2385596580.821918
train loss:2385596591.342466
train loss:2385596612.3835616
train loss:2385596603.6164384
train loss:2385596600.109589
train loss:2385596617.6438355
train loss:2385596607.1232877
train loss:2385596605.369863
train loss:2385596622.9041095
train loss:2385596600.109589
train loss:2385596593.0958905
train loss:2385596608.8767123
train loss:2385596635.178082
train loss:2385596594.849315
train loss:2385596596.60274
train loss:2385596610.630137
train loss:2385596628.1643834
train loss:2385596612.3835616
train loss:2385596601.8630137
train loss:2385596593.0958905
train loss:2385596633.4246573
train loss:2385596596.60274
train loss:2385596591.342466
train loss:2385596603.61643

In [62]:
# 验证模型
def test_model(model, dataloader):
    model.eval()

    total_loss = 0.
    for idx, (x, y) in enumerate(dataloader):
        y_pred = model(x)
        cur_loss = loss(y_pred, y)
        total_loss += cur_loss.item()
    print(f"Test loss: {total_loss/len(test_dataset)}")

In [63]:
num_epoch = 5
for i in range(num_epoch):
    print(f"==== Epoch {i} ====")
    train_model(model, train_dataloader)
    test_model(model, test_dataloader)

==== Epoch 0 ====
train loss:2385602140.9315066
Test loss: 2745565969.5342464
==== Epoch 1 ====
train loss:2385599142.5753427
Test loss: 2754330785.3150687
==== Epoch 2 ====
train loss:2385598176.4383564
Test loss: 2719609834.9589043
==== Epoch 3 ====
train loss:2385597746.849315
Test loss: 2808119772.9315066
==== Epoch 4 ====
train loss:2385597401.4246573
Test loss: 2896011797.0410957
